## **Labeled dataframes for all cases**

Después de aplicar la *skeletonization*, la arteria coronaria queda representada como una estructura muy delgada en forma de líneas de un solo vóxel de grosor dentro de un volumen 3D. Estos vóxeles están definidos en coordenadas del espacio de imagen, es decir, por índices enteros $(i, j, k)$ que indican su posición dentro de la rejilla del archivo NIfTI.

Para transformar estos vóxeles en puntos que representen su ubicación en el espacio físico del paciente (por ejemplo, en milímetros), el código realiza los siguientes pasos:

1. **Carga del archivo NIfTI del segmento**
   Se abre el archivo `.nii.gz` correspondiente a un segmento de arteria. Este archivo contiene una imagen binaria donde los vóxeles con valor > 0 pertenecen al esqueleto de ese segmento.

2. **Obtención del tamaño de vóxel**
   Se extrae del encabezado del archivo el tamaño físico de cada vóxel en milímetros. Por ejemplo, un vóxel puede medir $0.5 \times 0.5 \times 0.5$ mm.

3. **Extracción de coordenadas de vóxeles**
   Se buscan todos los índices $(i, j, k)$ de los vóxeles cuyo valor es mayor que 0, lo cual indica que forman parte del esqueleto del segmento.

4. **Centrado del vóxel**
   Como los índices hacen referencia a la esquina del vóxel, se suma $0.5$ a cada coordenada para centrar el punto en el centro del vóxel:

   $$
   (i + 0.5,\ j + 0.5,\ k + 0.5)
   $$

5. **Escalado a coordenadas reales**
   Se multiplican las coordenadas centradas por el tamaño del vóxel para pasar de índices de imagen a coordenadas físicas reales en milímetros:

   $$
   (Px, Py, Pz) = (i + 0.5, j + 0.5, k + 0.5) \times \text{voxel\_size}
   $$

6. **Creación de un DataFrame**
   Cada uno de estos puntos en coordenadas reales se guarda en un `DataFrame` junto con su etiqueta de segmento, resultando en una tabla con columnas: `['Px', 'Py', 'Pz', 'label']`.

Este proceso se repite para todos los segmentos de interés, generando un conjunto de puntos 3D etiquetados que representan el esqueleto arterial en el espacio real del paciente.

---

### **Imports**

In [ ]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix

### **Google Drive connection**

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Upload files**

In [ ]:
# Base input directory
input_base_dir = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/Processed/'

# Output directory
output_dir = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/Dataframes/'

### **Functions**

In [ ]:
def process_segments(input_dir, segments_to_process):
    """
    Processes multiple coronary artery segments from NIfTI files and returns
    a combined DataFrame with real-world coordinates and segment labels.

    Parameters:
    - input_dir (str): Directory containing the NIfTI files of the segments.
    - segments_to_process (list): List of segment labels to process.

    Returns:
    - pd.DataFrame: DataFrame with columns ['Px', 'Py', 'Pz', 'label'] containing
                    real-world coordinates and segment labels.
    """
    df_all_segments = pd.DataFrame(columns=["Px", "Py", "Pz", "label"])

    for segment_label in segments_to_process:
        segment_file = f'label_{segment_label}_intersection.nii.gz'
        segment_path = os.path.join(input_dir, segment_file)

        # Check if the file exists
        if not os.path.exists(segment_path):
            print(f"Warning: File {segment_file} not found in {input_dir}. Skipping.")
            continue

        # Load the NIfTI file
        nii_img = nib.load(segment_path)
        voxel_size = nii_img.header.get_zooms()
        skeleton = nii_img.get_fdata()

        # Extract voxel coordinates and convert to real-world coordinates
        voxel_coords = np.column_stack(np.where(skeleton > 0)).astype(float)
        real_coords = (voxel_coords + 0.5) * voxel_size

        # Create a DataFrame for the current segment
        df_segment = pd.DataFrame(real_coords, columns=["Px", "Py", "Pz"])
        df_segment["label"] = segment_label

        # Append to the main DataFrame
        df_all_segments = pd.concat([df_all_segments, df_segment], ignore_index=True)

    return df_all_segments

In [ ]:
# Function to process and save DataFrames for all cases
def process_and_save_all_cases(input_base_dir, output_dir, num_cases=20):
    """
    Processes RCA and LCA segments for all normal and diseased cases and saves the resulting DataFrames.

    Parameters:
    - input_base_dir (str): Base directory containing all cases.
    - output_dir (str): Directory to save the resulting DataFrames.
    - num_cases (int): Number of normal and diseased cases to process (default is 20).
    """

    for i in range(1, num_cases + 1):
        # Process normal cases
        input_dir_normal = os.path.join(input_base_dir, f'Normal_{i}')
        if os.path.exists(input_dir_normal):
            print(f"Processing Normal_{i}...")
            # Process RCA
            df_rca = process_segments(input_dir_normal, segments_rca)
            output_file_rca = os.path.join(output_dir, f'df_rca_n{i}.csv')
            df_rca.to_csv(output_file_rca, index=False)
            print(f"Saved RCA DataFrame for Normal_{i} to {output_file_rca}")

            # Process LCA
            df_lca = process_segments(input_dir_normal, segments_lca)
            output_file_lca = os.path.join(output_dir, f'df_lca_n{i}.csv')
            df_lca.to_csv(output_file_lca, index=False)
            print(f"Saved LCA DataFrame for Normal_{i} to {output_file_lca}")

        # Process diseased cases
        input_dir_diseased = os.path.join(input_base_dir, f'Diseased_{i}')
        if os.path.exists(input_dir_diseased):
            print(f"Processing Diseased_{i}...")
            # Process RCA
            df_rca = process_segments(input_dir_diseased, segments_rca)
            output_file_rca = os.path.join(output_dir, f'df_rca_d{i}.csv')
            df_rca.to_csv(output_file_rca, index=False)
            print(f"Saved RCA DataFrame for Diseased_{i} to {output_file_rca}")

            # Process LCA
            df_lca = process_segments(input_dir_diseased, segments_lca)
            output_file_lca = os.path.join(output_dir, f'df_lca_d{i}.csv')
            df_lca.to_csv(output_file_lca, index=False)
            print(f"Saved LCA DataFrame for Diseased_{i} to {output_file_lca}")


FUNCTIONS TO VISUALIZE (it was necessary to check that was correctly working)

In [ ]:
def calculate_mst(df):

    # Extract the coordinates of the points
    positions = df[['Px', 'Py', 'Pz']].values

    # Calculate the Euclidean distance matrix between all points
    dist_matrix = distance_matrix(positions, positions)

    # Create a graph with all nodes and distances as weights
    G = nx.Graph()
    num_nodes = len(positions)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            G.add_edge(i, j, weight=dist_matrix[i, j])

    # Get the Minimum Spanning Tree (MST)
    MST = nx.minimum_spanning_tree(G)

    return MST

In [ ]:
def visualize_mst(df, MST):

    # Extract the positions of the nodes
    positions = df[['Px', 'Py', 'Pz']].values

    # Get the positions for the nodes
    pos = {i: positions[i] for i in range(len(positions))}

    # Create lists for node and edge coordinates
    node_x, node_y, node_z = zip(*positions)

    edge_x = []
    edge_y = []
    edge_z = []
    for edge in MST.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Create the interactive visualization with Plotly
    fig = go.Figure()

    # Add edges (aristas)
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='blue', width=2),
        name='Edges'
    ))

    # Add nodes (nodos)
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Nodes'
    ))

    # Configure the layout for the visualization
    fig.update_layout(
        title='Minimum Spanning Tree (MST)',
        scene=dict(
            xaxis=dict(title='Px', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            yaxis=dict(title='Py', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            zaxis=dict(title='Pz', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            aspectmode='data'  # Ensure correct proportions between axes
        ),
        margin=dict(l=10, r=10, b=10, t=50),
        height=800  # Increase the plot size
    )

    # Display the plot
    fig.show()

In [ ]:
# Function to visualize segments
def visualize_segments(df):
    """
    Visualizes the segmented branches of the artery using a 3D scatter plot.
    Each segment is color-coded based on its assigned label.

    Parameters:
    df (pd.DataFrame): DataFrame containing artery segments with 'Px', 'Py', 'Pz', and 'label'.
    """
    # Get unique labels
    unique_labels = df['label'].unique()
    colors = np.linspace(0, 1, len(unique_labels))  # Generate color indices
    color_map = dict(zip(unique_labels, colors))  # Map each label to a color

    # Create figure
    fig = go.Figure()

    for label in unique_labels:
        label_data = df[df['label'] == label]

        fig.add_trace(go.Scatter3d(
            x=label_data['Px'],
            y=label_data['Py'],
            z=label_data['Pz'],
            mode='markers',
            marker=dict(size=3, color=color_map[label], colorscale='viridis'),
            name=f'Label {label}'
        ))

    # Configure layout
    fig.update_layout(
        title='Segmented Coronary Artery Visualization',
        scene=dict(
            xaxis=dict(title='Px', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            yaxis=dict(title='Py', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            zaxis=dict(title='Pz', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            aspectmode='data'
        ),
        margin=dict(l=10, r=10, b=10, t=50),
        height=800
    )

    fig.show()

In [ ]:
def load_and_visualize_example(output_dir, case_type, case_number, segment_type):
    """
    Loads a specific DataFrame and visualizes it to confirm successful processing.

    Parameters:
    - output_dir (str): Directory where the DataFrames are saved.
    - case_type (str): 'normal' or 'diseased'.
    - case_number (int): Case number (e.g., 1, 2, ... 20).
    - segment_type (str): 'rca' or 'lca'.
    """
    # Construct the file name
    file_name = f"df_{segment_type}_{case_type[0]}{case_number}.csv"
    file_path = os.path.join(output_dir, file_name)

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Error: File {file_name} not found in {output_dir}")
        return

    # Load the DataFrame
    df = pd.read_csv(file_path)
    print(f"Loaded DataFrame from {file_path}")
    # print(df.head())  # Display the first few rows for inspection

    # Visualize the segments
    print(f"Visualizing {segment_type.upper()} for {case_type.capitalize()} case {case_number}...")
    visualize_segments(df)
    return df

### **Saving dataframes**

In [ ]:
segments_rca = [1, 2, 3, 4, 16]
segments_lca = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17]

# Process all cases and save results
process_and_save_all_cases(input_base_dir, output_dir)

Processing Normal_1...


<ipython-input-4-119bceb92f2d>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_segments = pd.concat([df_all_segments, df_segment], ignore_index=True)


Saved RCA DataFrame for Normal_1 to /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_rca_n1.csv


<ipython-input-4-119bceb92f2d>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_segments = pd.concat([df_all_segments, df_segment], ignore_index=True)


Saved LCA DataFrame for Normal_1 to /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_lca_n1.csv
Processing Diseased_1...


...

Saved LCA DataFrame for Normal_20 to /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_lca_n20.csv
Processing Diseased_20...


<ipython-input-4-119bceb92f2d>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_segments = pd.concat([df_all_segments, df_segment], ignore_index=True)


Saved RCA DataFrame for Diseased_20 to /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_rca_d20.csv


<ipython-input-4-119bceb92f2d>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_segments = pd.concat([df_all_segments, df_segment], ignore_index=True)


Saved LCA DataFrame for Diseased_20 to /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_lca_d20.csv


### **Check the loading**

In [ ]:
# Example visualization: Normal case 1, RCA
df_rca_n1 = load_and_visualize_example(output_dir, case_type='normal', case_number=1, segment_type='rca')

Loaded DataFrame from /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_rca_n1.csv
Visualizing RCA for Normal case 1...


In [ ]:
df_rca_n1

,Px,Py,Pz,label
0,85.907307,76.754954,90.3125,1
1,85.907307,77.586986,90.9375,1
2,86.323323,77.170970,90.9375,1
3,86.323323,78.003002,91.5625,1
4,86.739339,78.419018,91.5625,1
...,...,...,...,...
488,149.557756,125.428828,27.8125,16
489,149.973772,125.428828,28.4375,16
490,150.389788,125.012812,28.4375,16
491,150.805805,125.012812,28.4375,16


In [ ]:
# Example visualization: Normal case 1, RCA
df_lca_d13 = load_and_visualize_example(output_dir, case_type='diseased', case_number=13, segment_type='lca')

Loaded DataFrame from /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Dataframes/df_lca_d13.csv
Visualizing LCA for Diseased case 13...


In [ ]:
df_lca_d13

,Px,Py,Pz,label
0,110.665949,105.142515,115.3125,5
1,111.060480,105.142515,115.3125,5
2,111.455011,105.142515,115.3125,5
3,111.849542,105.142515,115.3125,5
4,112.244073,105.142515,115.3125,5
...,...,...,...,...
880,168.267476,92.122991,48.4375,17
881,168.267476,92.517522,49.0625,17
882,168.267476,92.912053,49.0625,17
883,168.662007,83.837840,43.4375,17
